<a href="https://colab.research.google.com/github/superaja/covid19/blob/master/covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import requests
import io
import plotly.graph_objects as go
import time, datetime

In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# download all data

# NYC
def downloadPDF(state):
  y = datetime.datetime.now()
  if state == "CT":
    f_string = str(y.month)+str(y.day-1)+str(y.year)
    f_string
    file_name = "CTDPHCOVID19summary"+f_string+'.pdf'
    file_url = "https://portal.ct.gov/-/media/Coronavirus/"+file_name
    r = requests.get(file_url, stream = True) 
    with open('/content/drive/My Drive/Colab Notebooks/data/'+file_name,"wb") as pdf: 
      for chunk in r.iter_content(chunk_size=1024): 
        # writing one chunk at a time to pdf file 
        if chunk: 
          pdf.write(chunk) 
  else:
    download_files = ['covid-19-daily-data-summary.pdf', 'covid-19-daily-data-summary-deaths.pdf', 'covid-19-daily-data-summary-hospitalizations.pdf', 'covid-19-syndromic-surveillance.pdf'] 
    f_string = str(y.month)+str(y.day-1)+str(y.year) + '-' + str(y.hour)
    file_url = 'https://www1.nyc.gov/assets/doh/downloads/pdf/imm/'
    for fn in download_files:
      file_url = file_url+fn
      r = requests.get(file_url, stream = True) 
      with open('/content/drive/My Drive/Colab Notebooks/data/'+ f_string + '-'+ fn,"wb") as pdf: 
        for chunk in r.iter_content(chunk_size=1024): 
          # writing one chunk at a time to pdf file 
          if chunk: 
            pdf.write(chunk)
# JHU
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-30-2020.csv')
df_time_series = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_us_time_series = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')
#df_time_series = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
#df_time_series = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')


In [0]:
downloadPDF("NY")

In [77]:
df_time_series.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,22,22,24,24,40,40,74,84,94,110,110,120
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123,146,174,186,197,212
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60,74,87,90,139,201,230,264,302,367,409,454,511
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39,39,53,75,88,113,133,164,188,224,267,308,334
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,4,4,5,7


In [165]:
df_us_time_series.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [10]:
state_df = df_us_time_series[df_us_time_series['state'] == 'Washington']
state_df[state_df['date'] == state_df['date'].max()]

# Total US

latest_data = df_us_time_series[df_us_time_series['date'] == df_us_time_series['date'].max()]
total_us_confirmed = latest_data.groupby('state')['cases'].sum().sum()
total_us_deaths= latest_data.groupby('state')['deaths'].sum().sum()
latest_data['pct_confirmed'] = round((latest_data['cases'] / total_us_confirmed)*100, 2)
latest_data['pct_deaths'] = round((latest_data['deaths'] / total_us_deaths)*100, 2)
latest_data['pct_deaths_wrt_confirmed'] = round((latest_data['deaths'] /latest_data['cases'] )*100, 2)
latest_data.sort_values('pct_deaths', ascending=False)
top_10_confirmed = latest_data.nlargest(10, 'pct_confirmed')
top_10_deaths = latest_data.nlargest(10, 'pct_deaths')
top_10_deaths_wrt_cases = latest_data.nlargest(10, 'pct_deaths_wrt_confirmed')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [11]:
latest_data

,date,state,fips,cases,deaths,pct_confirmed,pct_deaths,pct_deaths_wrt_confirmed
1444,2020-03-29,Alabama,1,830,5,0.58,0.20,0.60
1445,2020-03-29,Alaska,2,114,1,0.08,0.04,0.88
1446,2020-03-29,Arizona,4,929,18,0.65,0.72,1.94
1447,2020-03-29,Arkansas,5,449,6,0.32,0.24,1.34
1448,2020-03-29,California,6,6266,130,4.41,5.23,2.07
1449,2020-03-29,Colorado,8,2315,47,1.63,1.89,2.03
1450,2020-03-29,Connecticut,9,1993,34,1.40,1.37,1.71
1451,2020-03-29,Delaware,10,232,6,0.16,0.24,2.59
1452,2020-03-29,District of Columbia,11,342,4,0.24,0.16,1.17
1453,2020-03-29,Florida,12,4942,59,3.48,2.37,1.19


In [12]:
fig = go.Figure()
fig.add_trace(go.Bar(x=top_10_confirmed['state'], y=top_10_confirmed['pct_confirmed'], name='% Confirmed', text=top_10_confirmed['pct_confirmed'], textposition='auto'))
fig.add_trace(go.Bar(x=top_10_confirmed['state'], y=top_10_confirmed['pct_deaths_wrt_confirmed'], name='Mortality Rate', text=top_10_confirmed['pct_deaths_wrt_confirmed'], textposition='auto'))
#fig.add_trace(go.Bar(x=top_10_deaths_wrt_cases['state'], y=top_10_deaths_wrt_cases['pct_deaths_wrt_confirmed'], name='% Deaths w.r.t Cases'))
fig.show()

In [0]:
# Data Prep
df_by_country = df.groupby('Country_Region').sum() # remove state/province
df_by_country['Country_Region'] = df_by_country.index

In [0]:
# add additional metrics
df_by_country['Not Yet Recovered'] = df_by_country['Confirmed'] - df_by_country['Deaths'] - df_by_country['Recovered']
df_by_country['% Recovered'] = round(df_by_country['Recovered'] / df_by_country['Confirmed'], 2)*100
df_by_country['% Mortality'] = round(df_by_country['Deaths'] / df_by_country['Confirmed'], 2)*100
df_by_country['% Not Yet Recovered'] = round(df_by_country['Not Yet Recovered'] / df_by_country['Confirmed'], 2)*100

In [0]:
# core KPIs

# Core Metrics by Country
def covid_core_metrics():
  country = ['Italy', 'US', 'Spain', 'China', 'Korea, South', 'United Kingdom', 'India']
  #country = list(df_by_country['Country/Region'].unique())
  x = df_by_country[df_by_country['Country_Region'].isin(country)]['Country_Region']
  y = df_by_country[df_by_country['Country_Region'].isin(country)]['% Recovered']
  y1 = df_by_country[df_by_country['Country_Region'].isin(country)]['% Mortality']
  y2 = df_by_country[df_by_country['Country_Region'].isin(country)]['% Not Yet Recovered']
  fig = go.Figure(data=go.Bar(x=x, y=y, name='% Recovered', text=y, textposition='auto', marker_color='green'))
  fig.add_trace(go.Bar(x=x, y=y1, name='% Mortality', text=y1, textposition='auto'))
  fig.add_trace(go.Bar(x=x, y=y2, name='% Not Yet Recovered', text=y2, textposition='auto', marker_color='rgb(144, 202, 249)'))
  fig.show()



In [0]:
# Confirmed Country Case Tracker
df_clean = df_time_series.drop(columns=['Province/State', 'Lat', 'Long'])
#country = ['Italy', 'US', 'Spain', 'China', 'Korea, South', 'United Kingdom', 'Germany']
country = ['Italy', 'US', 'Spain','United Kingdom']
def days_since_100_confirmed():
  fig = go.Figure()
  fig1 = go.Figure()
  fig_day_cases = go.Figure()
  for i in country:
    df_country = df_clean[df_clean['Country/Region'] == i]
    df_country = df_country.groupby('Country/Region').sum()
    country_confirmed = df_country.iloc[0].tolist()  
    country_confirmed_gt_100 = [i for i in country_confirmed[1:] if i>100]
    daily_confirmed = [country_confirmed_gt_100[j+1] - country_confirmed_gt_100[j] for j in range(len(country_confirmed_gt_100)-1)]
    label = [round(i/1000, 2) for i in country_confirmed_gt_100]
    days_since_100 = [i for i in range(1, len(country_confirmed_gt_100))]
    fig.add_trace(go.Scatter(x=days_since_100, y=country_confirmed_gt_100, mode='lines+markers', name=i+ ' Confirmed', \
                             textposition='top left'))
    fig1.add_trace(go.Scatter(x=days_since_100, y=country_confirmed_gt_100, mode='lines+markers', name=i+ ' Confirmed', \
                             textposition='top left'))
    fig_day_cases.add_trace(go.Bar(x=days_since_100, y=daily_confirmed, text=i, name=i + ' Confirmed'))
  
  fig.update_layout(title='Confirmed Cases by Country',
                    xaxis_title='Number of Days since 100 Confirmed Cases',
                    yaxis_title= 'Confirmed Cases',
                    )
  fig1.update_layout(title='Confirmed Cases by Country',
                    xaxis_title='Number of Days since 100 Confirmed Cases',
                    yaxis_title= 'Confirmed Cases - Log ',
                    yaxis_type='log'
                    )
  
  fig_day_cases.update_layout(title='Daily Confirmed Cases by Country',
                    xaxis_title='Number of Days since 100 Confirmed Cases',
                    yaxis_title= 'Daily Confirmed Cases',
                    )

  fig.show()
  fig1.show()
  fig_day_cases.show()

In [0]:
# Add States
#df_clean_US = df_time_series[df_time_series['Country/Region'] == 'US']
#df_clean_US = df_clean_US.drop(columns=['Lat', 'Long'])
#print (df_clean_US.head(5))
state = list(df_us_time_series['state'].unique())
# override states
state = ['Washington', 'California', 'Massachusetts', 'Florida', 'New York', 'New Jersey', 'Connecticut', 'Louisiana', 'Michigan']
def state_confirmed():
  fig = go.Figure()
  fig_log = go.Figure()
  fig_rate = go.Figure()
  for i in state:
    df_state = df_us_time_series[df_us_time_series['state']==i]
    df_state = df_state[df_state['cases']>500]
    df_state.reset_index(inplace=True)
    df_state.drop(columns=['index'])
    df_state.index = df_state.index + 1
    #state_confirmed = df_state.iloc[0].tolist()
    #state_confirmed_gt_100 = [i for i in state_confirmed[1:] if i>100]
    label = [round(m/1000, 2) for m in df_state['cases']]
    #days_since_100 = [i for i in range(1, len(state_confirmed_gt_100))]
    x = list(df_state.index)

    # Calculate Rate of Confirmation
    r, c = df_state.shape
    xy = []
    for j in range(r-1):
      temp = (df_state.iloc[j+1, 4] - df_state.iloc[j, 4] )/ df_state.iloc[j, 4]
      xy.append(temp)
    # Create Trace
    fig.add_trace(go.Scatter(x=x, y=df_state['cases'], mode='lines+markers', name=i, \
                             text=label, textposition='top left'))
    fig_rate.add_trace(go.Scatter(x=x, y=xy, mode='lines+markers', name=i, \
                             textposition='top left'))
    fig_log.add_trace(go.Scatter(x=x, y=df_state['cases'], mode='lines+markers', name=i, \
                             textposition='top left'))
  
  fig_log.update_layout(title='Confirmed Cases by State',
                    xaxis_title='Number of Days since 100 Confirmed Cases',
                    yaxis_title= 'Confirmed Cases - Log',
                    yaxis_type='log'
                    
                    )
  fig.update_layout(title='Confirmed Cases by State',
                  xaxis_title='Number of Days since 100 Confirmed Cases',
                  yaxis_title= 'Confirmed Cases'
                  )
  fig_rate.update_layout(title='Rate of Confirmed Cases by State',
                  xaxis_title='Number of Days since 100 Confirmed Cases',
                  yaxis_title= 'Rate of Confirmed Cases'
                  )
  
 
  
  fig.show()
  fig_log.show()
  fig_rate.show()

In [18]:
state_confirmed()

In [19]:
covid_core_metrics()
days_since_100_confirmed()

In [0]:
# pdf table extract
!pip install tabula-py

import tabula as tb

     |████████████████████████████████| 10.4MB 2.6MB/s 


In [0]:
# CT Data
df_ct = tb.read_pdf("https://portal.ct.gov/-/media/Coronavirus/CTDPHCOVID19summary3242020.pdf", pages='all', stream=True)[1]

# NYC Data
df_nyc = tb.read_pdf("https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary.pdf", pages='all', stream=True)[0]

In [0]:
country = ['US', 'Italy', 'Spain']
x = df_by_country[df_by_country['Country/Region'].isin(country)]['Country/Region']
y = df_by_country[df_by_country['Country/Region'].isin(country)]['% Recovered']

In [0]:
df_time_series.drop(columns=['Lat', 'Long'], inplace=True)

In [0]:
countries = list(df['Country/Region'].unique())
us_states = list(df[df['Country/Region']=='US']['Province/State'].unique())

In [0]:
def rate_calc(y):
  rate = [0]
  i=0
  #while i < len(y)-1:
  for i in range(len(y)-1):
    if y[i] == 0:
      temp_rate = 0
    else: 
      temp_rate = round((y[i+1] - y[i]) / y[i], 1)*100
    rate.append(temp_rate)
    i = i+1

  rate_label = [str(i) + '%' for i in rate]
  return rate, rate_label
#format 

#y = [round(i/1000, 0) for i in y]
#y_us = [round(i/1000, 0) for i in y_us]
#y_china = [round(i/1000, 0) for i in y_china]